# Phase2 Functional Model

## Download files

In [ ]:
# Upload model
from google.colab import files
uploaded = files.upload()

In [ ]:
!unzip -qq necessary_files.zip
%cd necessary_files

--2021-06-11 06:37:30--  https://s3-us-west-2.amazonaws.com/static.pyimagesearch.com/real-time-object-detection-deep-learning/real-time-object-detection.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.218.72
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.218.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21376463 (20M) [application/zip]
Saving to: ‘real-time-object-detection.zip’

real-time-object-de 100%[===================>]  20.39M  10.3MB/s    in 2.0s    

2021-06-11 06:37:33 (10.3 MB/s) - ‘real-time-object-detection.zip’ saved [21376463/21376463]

/content/real-time-object-detection


In [ ]:
# Upload Video
from google.colab import files
uploaded = files.upload()

Saving Subway.mp4 to Subway.mp4


### Import Packages

In [ ]:
# import the necessary packages
from matplotlib import pyplot as plt
from imutils.video import FPS
import numpy as np
import argparse
import imutils
import cv2

In [ ]:
#@markdown # Declaration of initial parameters
#@markdown * Input video
#@markdown * Output video
#@markdown * probability for model
#@markdown * Model
#@markdown * Confidence
args = {
	"input" : "Subway.mp4",
	"output" : "output.avi",
	"prototxt": "MobileNetSSD_deploy.prototxt.txt",
	"model": "MobileNetSSD_deploy.caffemodel",
	"confidence" : 0.5
}

In [ ]:
# initialize the list of class labels MobileNet SSD was trained to
# detect, then generate a set of bounding box colors for each class
CLASSES = ["background", "aeroplane", "bicycle", "bird", "boat",
	"bottle", "bus", "car", "cat", "chair", "cow", "diningtable",
	"dog", "horse", "motorbike", "person", "pottedplant", "sheep",
	"sofa", "train", "tvmonitor"]
COLORS = np.random.uniform(0, 255, size=(len(CLASSES), 3))

# load our serialized model from disk
print("[INFO] loading model...")
net = cv2.dnn.readNetFromCaffe(args["prototxt"], args["model"])

[INFO] loading model...


In [ ]:
# grab a reference to the video file, initialize pointer to output
# video file, and initialize the FPS counter
print("[INFO] opening video file...")
vs = cv2.VideoCapture(args["input"])
writer = None
fps = FPS().start()

[INFO] opening video file...


## Helpers

### Classes

In [ ]:
# Define rectangle based on two dots
class rectangle:
    def __init__(self, x1, y1, x2, y2):
        self.x1 = x1
        self.y1 = y1
        self.x2 = x2
        self.y2 = y2
      
    def coordinates(self):
      return self.x1, self.y1, self.x2, self.y2

### Functions

In [ ]:
# Determine if 2 rectangles are overlaped
def overlap(r1, r2):
    # Verifing first rectangle dots
    if (r1.x2>=r2.x1 and r1.x2 <=r2.x2 and r1.y2>=r2.y1 and r1.y2 <=r2.y2):
      return True
    if (r1.x1>=r2.x1 and r1.x1 <=r2.x2 and r1.y1>=r2.y1 and r1.y1 <=r2.y2):
      return True
    if (r1.x2>=r2.x1 and r1.x2 <=r2.x2 and r1.y1>=r2.y1 and r1.y1 <=r2.y2):
      return True
    if (r1.x1>=r2.x1 and r1.x1 <=r2.x2 and r1.y2>=r2.y1 and r1.y2 <=r2.y2):
      return True
    
    # Verifing second rectangle dots
    if (r2.x2>=r1.x1 and r2.x2 <=r1.x2 and r2.y2>=r1.y1 and r2.y2 <=r1.y2):
      return True
    if (r2.x1>=r1.x1 and r2.x1 <=r1.x2 and r2.y1>=r1.y1 and r2.y1 <=r1.y2):
      return True
    if (r2.x2>=r1.x1 and r2.x2 <=r1.x2 and r2.y1>=r1.y1 and r2.y1 <=r1.y2):
      return True
    if (r2.x1>=r1.x1 and r2.x1 <=r1.x2 and r2.y2>=r1.y1 and r2.y2 <=r1.y2):
      return True
    return False

# Count overlaped rectangles
def count_cases(rectangles):
    overlaped = 0
    for i in range(1,len(rectangles)):
        for j in range(1,len(rectangles)):
            if overlap(rectangles[0],rectangles[j]):
                overlaped +=1
                break
        rectangles.pop(0)    

    return overlaped


## Predict video

In [ ]:
total_cases = [] # Total overlapted rectangles
total_rectangles = [] # Total rectangles

# loop over the frames from the video stream
while True:
  
	# grab the next frame
	frame = vs.read()[1]

	# if we did not grab a frame then we have reached the end of the
	# video
	if frame is None:
		break
	
	# resize the frame to have a maximum width of 400 pixels
	frame = imutils.resize(frame, width=400)

	# grab the frame dimensions and convert it to a blob
	(h, w) = frame.shape[:2]
	blob = cv2.dnn.blobFromImage(frame, 0.007843, (300, 300), 127.5)

	# pass the blob through the network and obtain the detections and
	# predictions
	net.setInput(blob)
	detections = net.forward()
	rectangles = []
  

 	# loop over the detections
	for i in np.arange(0, detections.shape[2]):
		# extract the confidence (i.e., probability) associated with
		# the prediction
		confidence = detections[0, 0, i, 2]

		# filter out weak detections by ensuring the `confidence` is
		# greater than the minimum confidence
		if confidence > args["confidence"]:
			# extract the index of the class label from the
			# `detections`, then compute the (x, y)-coordinates of
			# the bounding box for the object
			idx = int(detections[0, 0, i, 1])
			box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
			(startX,startY,endX,endY) = box.astype("int")
			rectangles.append(rectangle(startX,startY,endX,endY))
			# draw the prediction on the frame
			label = "{}: {:.2f}%".format(CLASSES[idx],
				confidence * 100)
			cv2.rectangle(frame,(startX, startY), (endX, endY), COLORS[idx], 2)
			y = startY - 15 if startY - 15 > 15 else startY + 15
		# Variable used for add rectangles to $total_rectangles
		aux = []
		for i,j in enumerate(rectangles):
			# print(i)
			# print(j.coordinates())
			aux.append(j)

	total_rectangles.append(aux)
	overlaped = count_cases(rectangles)
	total_cases.append(overlaped)
	# Fucntion used for drawing a text with cases on every frame
	frame = cv2.putText(frame, 'Cases:'+str(overlaped), (25,25), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 1, cv2.LINE_AA)
	# update the FPS counter
	fps.update()

	# if the video writer is None *AND* we are supposed to write
	# the output video to disk initialize the writer
	if writer is None and args["output"] is not None:
		fourcc = cv2.VideoWriter_fourcc(*"MJPG")
		writer = cv2.VideoWriter(args["output"], fourcc, 20,
			(frame.shape[1], frame.shape[0]), True)
  
	# if the writer is not None, write the frame with recognized
	# faces to disk
	if writer is not None:
		writer.write(frame)
  
# stop the timer and display FPS information
fps.stop()
print("[INFO] elapsed time: {:.2f}".format(fps.elapsed()))
print("[INFO] approx. FPS: {:.2f}".format(fps.fps()))

# do a bit of cleanup
vs.release()

# check to see if the video writer point needs to be released
if writer is not None:
	writer.release()

In [ ]:
# converting video from .avi to .mp4
!ffmpeg -i output.avi output.mp4

ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib

### Display video inline

In [ ]:
# Additional libraries
from IPython.display import HTML
from base64 import b64encode

# Open video
mp4 = open("output.mp4", "rb").read()

# Play video using HTMl
dataURL = "data:videoP/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=400 controls>
      <source src="%s" type="video/mp4">
</video>
""" % dataURL)